In [24]:
import pandas as pd
import os
import requests as req
import json
import numpy as np
import datetime
from matplotlib import style
import matplotlib
%matplotlib inline
style.use('ggplot')

# This is to import custom-made modules

# This can be removed after making these modules a real library
import os, sys
#lib_path = os.path.abspath(os.path.join('..', 'src')) # relative path of the source code in Box Folder 
#sys.path.append(lib_path)

In [32]:
data = pd.read_csv(os.path.join(".","Units-1311.csv")) 

In [33]:
data['opponent'].unique()

array(['Z', 'T', 'P'], dtype=object)

In [34]:
#data2 = pd.read_csv(os.path.join("..","lossreplay.csv")) 

data.replace({'opponent' : { 'T' : 1, 'Z' : 2, 'P' : 3 }}, inplace =True)


In [40]:
data.head(100)

,winner,opponent,seconds,BarracksTechLab,Hellion,MULE,VikingFighter,Barracks,SiegeTankSieged,Ghost,...,SupplyDepotLowered,MissileTurret,Starport,Nuke,StarportTechLab,StarportReactor,SensorTower,Marauder,BarracksReactor,PointDefenseDrone
0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
datatoTest = data.iloc[12:95, 1:] #split this up to test (first game)
datatoModel = data.iloc[96:, :] #

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
X = datatoModel.iloc[:,1:]
y = datatoModel['winner']

In [7]:
#X = data.iloc[:,1:]
#y  = data['winner']

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

In [63]:
from sklearn.neighbors import KNeighborsClassifier

In [64]:
knn = KNeighborsClassifier(n_neighbors = 10)

In [65]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [66]:
knn.score(X_test, y_test)

0.8086727751065953

In [67]:
X_test.head()

,opponent,seconds,BarracksTechLab,Hellion,MULE,VikingFighter,Barracks,SiegeTankSieged,Ghost,SupplyDepot,...,SupplyDepotLowered,MissileTurret,Starport,Nuke,StarportTechLab,StarportReactor,SensorTower,Marauder,BarracksReactor,PointDefenseDrone
111502,2,224,0,0,1,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
50244,2,1229,2,0,0,0,15,28,0,12,...,25,6,2,0,1,1,0,0,5,0
1827,3,709,3,0,0,0,7,2,0,1,...,19,0,2,0,0,2,0,6,2,0
411513,2,724,0,12,4,0,1,0,0,6,...,10,0,3,0,1,1,0,0,0,0
466583,1,929,3,0,7,0,6,16,0,10,...,19,14,0,3,0,0,2,2,2,0


In [68]:
y_test.head()

111502    0
50244     1
1827      0
411513    1
466583    1
Name: winner, dtype: int64

In [69]:
X_test.shape

(121253, 54)

In [70]:
#forMatt = data.iloc[2:50,1:]

In [71]:
predictions = knn.predict_proba(datatoTest)

In [72]:
predictions[0][1]

0.4

In [73]:
predictions[2][1]

0.6

In [74]:
len(predictions)

83

In [75]:
mylist =[]
for i in xrange(len(predictions)):
    mylist.append(predictions[i][1])

In [76]:
datatoTest['prediction'] = mylist

In [85]:
#this is the average probability of the player winning the game (which in this case is true)
reduce(lambda x, y: x + y, mylist) / len(mylist)

0.38674698795180734

In [77]:
toplot = pd.DataFrame()
toplot['seconds'] = datatoTest['seconds']
toplot['prediction'] = datatoTest['prediction']
toplot.head()

,seconds,prediction
12,0,0.4
13,20,0.7
14,37,0.6
15,54,0.4
16,63,0.2


In [80]:
toplot.to_csv('forMatt.csv', index=False, header=False)